# About This Notebook

This data was downloaded from Kaggle.com (https://www.kaggle.com/datasets/knightbearr/sales-product-data?resource=download). It is sales data from a fictional company during 2019. We will call this company Acme, Inc.

# Data Description

• Order ID - An Order ID is the number system that Amazon uses exclusively to keep track of orders. Each order receives its own Order ID that will not be duplicated. This number can be useful to the seller when attempting to find out certain details about an order such as shipment date or status.

• Product - The product that have been sold.

• Quantity Ordered - Ordered Quantity is the total item quantity ordered in the initial order (without any changes).

• Price Each - The price of each products.

• Order Date - This is the date the customer is requesting the order be shipped.

• Purchase Address - The address from which the order was purchased.

# Importing the necessary libraries



In [84]:
# Basic utilities and settings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='statsmodels')
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format
import numpy as np
import matplotlib.pyplot as plt

# Collections and datetime utilities
from collections import Counter

# Scikit-learn utilities and models
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# Keras and TensorFlow
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Statsmodels utilities and models
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

# Importing the Data

In [66]:
df = pd.read_csv('https://raw.githubusercontent.com/bbrauser/SalesAnalysis/main/Sales_Data.csv')

df['Sales'] = df['Quantity Ordered'] * df['Price Each']

df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Sales
0,176558,USB-C Charging Cable,2,12,04/19/19 08:46,"917 1st St, Dallas, TX 75001",24
1,176559,Bose SoundSport Headphones,1,100,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",100
2,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",600
3,176560,Wired Headphones,1,12,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",12
4,176561,Wired Headphones,1,12,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",12
...,...,...,...,...,...,...,...
433311,156419,27in 4K Gaming Monitor,1,390,02/18/19 13:31,"827 8th St, New York City, NY 10001",390
433312,156420,Macbook Pro Laptop,1,"1,700",02/16/19 09:24,"488 Lake St, San Francisco, CA 94016","1,700"
433313,156421,AAA Batteries (4-pack),1,3,02/18/19 22:01,"174 8th St, Atlanta, GA 30301",3
433314,156422,iPhone,1,700,02/02/19 14:28,"746 South St, Austin, TX 73301",700


# Cleaning the Data

In [67]:
# Remove all NaN values from the dataset
df.dropna(inplace=True)

# Use convert_dtypes for an automatic conversion (it may not convert 'Order Date', so we'll still convert it manually)
df = df.convert_dtypes()

df['Order Date'] = pd.to_datetime(df['Order Date'])

In [68]:
# Function to convert the list of products into the desired format
def convert_product_list(product_list):
    counts = Counter(product_list)
    return ', '.join([f"{v}x {k}" for k, v in counts.items()])

# Apply the function to the 'Product' column
df['All Products'] = df['Product'].apply(convert_product_list)

# Sales Forecasting

## What will be the total sales for January 2020?

### Data preprocsessing

In [69]:
# Grouping rows by Order ID and combining products into a list
jan_2020_df = df.groupby('Order ID').agg({
    'Product': list,
    'Sales': 'sum',
    'Order Date': 'first',
    'Purchase Address': 'first'
}).reset_index()

### Scaling data

In [70]:
sales_data = jan_2020_df.resample('M', on='Order Date').sum(numeric_only=True)['Sales']
scaler = MinMaxScaler()
sales_data_scaled = scaler.fit_transform(sales_data.values.reshape(-1, 1))

### Data transformation

In [71]:
look_back = 3
X, Y = [], []

for i in range(len(sales_data_scaled) - look_back):
    X.append(sales_data_scaled[i:(i + look_back)])
    Y.append(sales_data_scaled[i + look_back])

X = np.array(X)
Y = np.array(Y)

### Model building

In [72]:
model = Sequential()
model.add(LSTM(50, input_shape=(X.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X, Y, epochs=100, batch_size=1)

Epoch 1/100
10/10 [==============================] - 2s 4ms/step - loss: 0.2926
Epoch 2/100
10/10 [==============================] - 0s 3ms/step - loss: 0.2080
Epoch 3/100
10/10 [==============================] - 0s 3ms/step - loss: 0.1374
Epoch 4/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0980
Epoch 5/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0825
Epoch 6/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0779
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0751
Epoch 8/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0741
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0736
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0745
Epoch 11/100
10/10 [==============================] - 0s 4ms/step - loss: 0.0732
Epoch 12/100
10/10 [==============================] - 0s 3ms/step - loss: 0.0723
Epoch 13/100
10/10 [=================

### Prediction

In [73]:
last_known_data = sales_data_scaled[-look_back:]
predicted_sales = model.predict(last_known_data.reshape(1, look_back, 1))
predicted_sales = scaler.inverse_transform(predicted_sales)
print(f"The total sales for January 2020 is predicted to be ${format(predicted_sales[0][0], ',.2f')}.")

1/1 [==============================] - 0s 461ms/step
The total sales for January 2020 is predicted to be $3,992,506.00.


## Which products are expected to be the top sellers in January, February, and March 2020?

### Data preprocsessing

In [74]:
jan_feb_mar_2020_df = df

# Extract month and year for filtering
jan_feb_mar_2020_df['Year'] = jan_feb_mar_2020_df['Order Date'].dt.year
jan_feb_mar_2020_df['Month'] = jan_feb_mar_2020_df['Order Date'].dt.month

### Scaling the data

In [75]:
scaler = MinMaxScaler()
jan_feb_mar_2020_df['Quantity Ordered'] = scaler.fit_transform(jan_feb_mar_2020_df[['Quantity Ordered']])

### Data transformation

In [76]:
product_time_series = jan_feb_mar_2020_df.groupby(['Order Date', 'Product']).sum(numeric_only=True)['Quantity Ordered'].unstack()
product_time_series = product_time_series.resample('M').sum().fillna(0)

### Model building

In [77]:
# Choose the top-selling product for demonstration
top_product = df.groupby('Product').sum(numeric_only=True)['Quantity Ordered'].idxmax()

# Train ARIMA model
model = ARIMA(product_time_series[top_product], order=(5,1,0))
model_fit = model.fit()

### Prediction

In [78]:
forecast = model_fit.forecast(steps=3)
print(f"Predicted sales for {top_product} in:")
print(f"January 2020: ${forecast[0].round(2)}")
print(f"February 2020: ${forecast[1].round(2)}")
print(f"March 2020: ${forecast[2].round(2)}")

Predicted sales for AAA Batteries (4-pack) in:
January 2020: $353.66
February 2020: $157.2
March 2020: $449.33


# Inventory Management

## Based on historical sales, how much inventory should we stock for each product in January 2020?

### Data Preprocessing

In [87]:
jan_inventory_2020 = df

jan_inventory_2020['Order Date'] = pd.to_datetime(jan_inventory_2020['Order Date'])
jan_inventory_2020 = jan_inventory_2020[jan_inventory_2020['Order Date'] <= '2019-12-31']
jan_inventory_2020['Month'] = jan_inventory_2020['Order Date'].dt.month
jan_inventory_2020['Year'] = jan_inventory_2020['Order Date'].dt.year

<ipython-input-87-cbcb27b7840b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_inventory_2020['Month'] = jan_inventory_2020['Order Date'].dt.month
<ipython-input-87-cbcb27b7840b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_inventory_2020['Year'] = jan_inventory_2020['Order Date'].dt.year


### Scalling the data

In [80]:
scaler = MinMaxScaler()
monthly_product_sales['Scaled Quantity'] = scaler.fit_transform(monthly_product_sales[['Quantity Ordered']])

### Data Transformation

In [81]:
X, y = [], []

for product in monthly_product_sales['Product'].unique():
    product_data = monthly_product_sales[monthly_product_sales['Product'] == product]['Scaled Quantity'].values
    for i in range(3, len(product_data)):
        X.append(product_data[i-3:i])
        y.append(product_data[i])

X, y = np.array(X), np.array(y)

### Model Building

In [82]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(3, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

### Prediction

In [86]:
predictions = {}

for product in monthly_product_sales['Product'].unique():
    product_data = monthly_product_sales[(monthly_product_sales['Product'] == product) & (monthly_product_sales['Year'] == 2019) & (monthly_product_sales['Month'] >= 10)]['Scaled Quantity'].values
    if len(product_data) == 3:  # Ensure we have all three months data
        scaled_prediction = model.predict(product_data.reshape(1, 3, 1))
        prediction = scaler.inverse_transform(scaled_prediction)
        predictions[product] = prediction[0][0]

predictions

1/1 [==============================] - 0s 133ms/step


{'20in Monitor': -0.6785455,
 '27in 4K Gaming Monitor': -0.61500907,
 '27in FHD Monitor': 0.14638817,
 '34in Ultrawide Monitor': -0.4937882,
 'AA Batteries (4-pack)': 261.6098,
 'AAA Batteries (4-pack)': 105.01117,
 'Apple Airpods Headphones': 2.5910556,
 'Bose SoundSport Headphones': 3.9297602,
 'Flatscreen TV': -0.40238592,
 'Google Phone': -0.94587106,
 'LG Dryer': -1.039228,
 'LG Washing Machine': -1.039228,
 'Lightning Charging Cable': 44.34246,
 'Macbook Pro Laptop': -1.039228,
 'ThinkPad Laptop': -0.9566084,
 'USB-C Charging Cable': 60.26204,
 'Vareebadd Phone': -1.039228,
 'Wired Headphones': 46.03058,
 'iPhone': -0.86533993}

## Which products have a higher likelihood of running out of stock in January 2020?

### Data preprocessing

In [ ]:
# Filter only January sales from past years
january_sales = old_df[old_df['Order Date'].dt.month == 1]

# Group by product to get total sales for each product in January
product_sales_january = january_sales.groupby('Product').agg({'Quantity Ordered': 'sum'}).reset_index()

### Data Scaling

In [ ]:
X = product_sales_january['Product']
y = product_sales_january['Quantity Ordered']

# Encoding the product names (categorical data)
X = pd.get_dummies(X)

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Model Building

In [ ]:
# Initialize the model and train it
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Check the performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

### Prediction

In [ ]:
january_predictions = model.predict(X.values)
product_sales_january['Predicted Quantity for Next January'] = january_predictions

# Filter out products with higher likelihood of running out (based on a threshold or top N products)
# For this example, I'm choosing products that have predicted sales higher than a certain threshold
threshold = january_predictions.mean() + 1.5 * january_predictions.std()
at_risk_products = product_sales_january[product_sales_january['Predicted Quantity for Next January'] > threshold]

at_risk_products

# Geographical Analysis

## Can we predict sales surges for certain products based on seasonal trends? (e.g., higher sales of certain items during holidays or specific seasons).

### Data Preprocessing

In [ ]:
# Extract city
old_df['City'] = old_df['Purchase Address'].str.extract(r'([^,]+), \w{2}\s*\d*$', expand=False)

# Aggregate sales by city and month
old_df['Month'] = old_df['Order Date'].dt.month
monthly_sales_per_city = old_df.groupby(['City', 'Month'])['Order Amount'].sum(numeric_only=True).reset_index()

### Linear Regression Model

In [ ]:
# Fit linear regression for each city

# Create an empty dictionary to store predictions
predictions = {}

for city in monthly_sales_per_city['City'].unique():
    city_data = monthly_sales_per_city[monthly_sales_per_city['City'] == city]
    X = city_data['Month'].values.reshape(-1, 1)
    y = city_data['Order Amount'].values

    model = LinearRegression()
    model.fit(X, y)

    # Predict sales for the next month (assuming the next month is after the max month in your data)
    next_month = city_data['Month'].max() + 1
    predicted_sales = model.predict(np.array([[next_month]]))[0]

    predictions[city] = predicted_sales

### Sort and Display

In [ ]:
# 3.1 Rank cities by predicted sales
sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

# 3.2 Display top cities
for city, predicted_sale in sorted_predictions:
    formatted_sale = "${:,.2f}".format(predicted_sale)
    print(f"{city} - Predicted Sales: {formatted_sale}")

## Are there cities where certain products are likely to be more popular in the future?

### Data Preparation

In [ ]:
old_df['City'] = old_df['Purchase Address'].apply(lambda x: x.split(',')[1].strip())

### Historical Analysis

In [ ]:
product_city_counts = old_df.groupby(['City', 'Product']).size().unstack(fill_value=0)

### Trend Identification

In [ ]:
old_df['Month'] = old_df['Order Date'].dt.month
monthly_product_city_counts = old_df.groupby(['City', 'Month', 'Product']).size().unstack(fill_value=0)

### Predictions

In [ ]:
predictions = {}

for city in monthly_product_city_counts.index.get_level_values(0).unique():
    for product in monthly_product_city_counts.columns:
        X = monthly_product_city_counts.loc[city].index.values.reshape(-1, 1)
        y = monthly_product_city_counts.loc[city][product].values
        model = LinearRegression().fit(X, y)
        next_month = X[-1][0] + 1
        prediction = model.predict(np.array([[next_month]]))[0]
        predictions[(city, product)] = prediction

### Result Interpretation

In [ ]:
increasing_products_by_city = {}

for (city, product), prediction in predictions.items():
    if prediction > monthly_product_city_counts.loc[city][product].mean():
        if city not in increasing_products_by_city:
            increasing_products_by_city[city] = []
        increasing_products_by_city[city].append(product)

# Convert the nested dictionary to a list of dictionaries
data_list = [{'City': city, 'Products': ', '.join(products)}
             for city, products in increasing_products_by_city.items()]

# Flatten the dictionary into a list of (city, product) pairs
flat_data = [(city, product) for city, products in increasing_products_by_city.items() for product in products]

# Convert the list to a DataFrame
multi_index_df = pd.DataFrame(flat_data, columns=['City', 'Product'])

# Set the 'City' column as the index
multi_index_df.set_index('City', inplace=True)

multi_index_df